## 池化层
- 卷积对位置十分敏感，就比如检测垂直边缘，如果像素发生了偏移那么就检测不出来边缘了, **1像素的位移就会导致0输出**
- 需要一定程度的平移不变性

In [8]:
import torch
from d2l import torch as d2l 
from torch import nn
import torch.nn.functional as F
X = torch.zeros((4, 5))
X[:, :2] = 1
K = torch.tensor([[1, -1]])
d2l.corr2d(X, K)

tensor([[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

- 池化层的本质就是忽略一些细节，以上述例子为例，如果加入一个2 * 2的池化层，那么就能容忍左右各1个的像素位置偏移
- 最大池化层：每个窗口中的最强的模式信号
- 平均池化层：最大->平均
- 缓解卷积层对位置的敏感性

- 自定义2d池化

In [9]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i:i + p_h, j:j + p_w].max()
            if mode == 'avg':
                Y[i, j] = X[i:i + p_h, j:j + p_w].mean()
    return Y

X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [10]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

- Pool层也有填充和步幅

In [13]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X, X.shape

(tensor([[[[ 0.,  1.,  2.,  3.],
           [ 4.,  5.,  6.,  7.],
           [ 8.,  9., 10., 11.],
           [12., 13., 14., 15.]]]]),
 torch.Size([1, 1, 4, 4]))

In [16]:
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

- 手动设定填充和步幅

In [17]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [18]:
pool2d = nn.MaxPool2d((2, 3), padding=(0, 1), stride=(2, 3))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

- 多通道

In [19]:
X = torch.cat([X, X + 2], dim=1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 2.,  3.,  4.,  5.],
          [ 6.,  7.,  8.,  9.],
          [10., 11., 12., 13.],
          [14., 15., 16., 17.]]]])

In [20]:
pool2d = nn.MaxPool2d(3, stride=2, padding=1)
pool2d(X) 

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 7.,  9.],
          [15., 17.]]]])